In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import re
import time

### NASA Mars News

In [2]:
# url to scrape
mars_news_url = 'https://mars.nasa.gov/news/'

In [3]:
!which chromedriver

/usr/local/bin/chromedriver


In [34]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
browser.visit(mars_news_url)

In [6]:
titles_list = []
articles_list = []
for x in range(1,4):
    html = browser.html
    soup = bs(html, 'lxml')
    articles = soup.find_all('li', class_='slide')
    for z in articles:
        titles_list.append(z.find('h3').text.strip())
        articles_list.append(z.find('div', class_='article_teaser_body').text.strip())
    try:
        links_found = browser.links.find_by_text('More').click()
    except:
        print(f'No page {x+1}')


    

In [7]:
articles_list

['The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.',
 "The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan.",
 "Learn more about the agency's next Red Planet mission during a live event on June 17.",
 "This new portrait of the Red Planet's neighbors was taken during a time when there's more dust in the air on Mars.",
 'Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.',
 "Taken with the infrared camera aboard NASA's Odyssey orbiter, they reveal temperature variations on the small moon as it drifts into and out of Mars’ shadow.",
 'Two astronauts collected Moon rocks on Apollo 11. It will take three robotic systems working together to gather up the 

In [8]:
mars_news = pd.DataFrame({
    'Titles': titles_list,
    'Articles': articles_list
})

In [9]:
mars_news

,Titles,Articles
0,How NASA's Mars Helicopter Will Reach the Red ...,The small craft will seek to prove that powere...
1,The Launch Is Approaching for NASA's Next Mars...,The Red Planet's surface has been visited by e...
2,NASA to Hold Mars 2020 Perseverance Rover Laun...,Learn more about the agency's next Red Planet ...
3,"While Stargazing on Mars, NASA's Curiosity Rov...",This new portrait of the Red Planet's neighbor...
4,NASA's Mars Rover Drivers Need Your Help,Using an online tool to label Martian terrain ...
...,...,...
115,NASA's Mars Reconnaissance Orbiter Undergoes M...,Other orbiters will continue relaying data fro...
116,NASA's Mars 2020 Rover Goes Coast-to-Coast to ...,The agency's first step in returning rocks fro...
117,NASA Prepares for Moon and Mars With New Addit...,Robotic spacecraft will be able to communicate...
118,NASA Administrator Statement on Moon to Mars I...,Jim Bridenstine addresses NASA's ambitious pla...


### JPL Mars Space Images - Featured Image

In [13]:
mars_images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [31]:
browser.visit(mars_images_url)

In [32]:
image_url = ''
try:
    browser.find_by_id('full_image').click()
    html = browser.html
    soup = bs(html, 'lxml')
    image_url = soup.find('img', class_='fancybox-image')
except:
    print('Could not get full image')
image_url = f'https://www.jpl.nasa.gov{image_url["src"]}'

### Mars Weather

In [35]:
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'

In [45]:
browser.visit(mars_weather_url)

In [46]:
html = browser.html

In [47]:
soup = bs(html, 'lxml')

In [39]:
mars_weather = soup.find('article', class_='css-1dbjc4n r-1loqt21 r-18u37iz r-1ny4l3l r-o7ynqc r-6416eg')

In [48]:
mars_weather = (
    soup.
    find('article', class_='css-1dbjc4n r-1loqt21 r-18u37iz r-1ny4l3l r-o7ynqc r-6416eg').
    find_all('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")[4].
    text.
    strip()
)

In [49]:
mars_weather_found = re.search("low .+ high .+ºF\)", mars_weather)

In [56]:
mars_weather = mars_weather_found[0] if mars_weather_found else 'no weather found'
mars_weather = re.sub("ºF", " Farenheit", mars_weather)
mars_weather = re.sub("ºC", " Celsius", mars_weather)

'low -89.5 Celsius (-129.1 Farenheit) high -3.9 Celsius (25.0 Farenheit)'

### Mars Facts

In [ ]:
mars_facts_url = 'https://space-facts.com/mars/'

In [ ]:
mars_facts_table = pd.read_html(mars_facts_url)

In [ ]:
mars_facts_table_df = mars_facts_table[0]

In [ ]:
mars_facts_table_df.columns = ['Property', 'Measures']

In [ ]:
mars_facts_table_df.to_html('table.html', index=False)

### Mars Hemispheres

In [ ]:
mars_hemipsheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [ ]:
browser.visit(mars_hemipsheres_url)

In [ ]:
html = browser.html

In [ ]:
mars_images_dict = []
idx = -1;
for _ in browser.find_by_css(".description"):
    try:
        img = browser.find_by_css(".description")
        idx += 1
        link = img[idx].find_by_css(".itemLink").first
        title = link.text
        link.click()
        html = browser.html
        url = browser.find_by_text(' (jpg) 1024px wide').first
        href = bs(url.html, 'lxml').find('a')['href']
        print(title)
        mars_images_dict.append({"title": title, "img_url": href})
        browser.back()
        time.sleep(0.5)
    except:
        print('Anchor not found')
    